# Урок 6. Многоклассовая классификация.

Посмотрим на примере алгоритма логистической регрессии и метода опорных векторов, как работать с различными методами многоклассовой классификации.

### 1.
Вспомните датасет Wine. Загрузите его, разделите на тренировочную и тестовую выборки (random_state=17), используя только [9, 11, 12] признаки.

In [1]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

In [9]:
wine_data = load_wine()
X = wine_data.data[:,[9, 11, 12]]
y = wine_data.target
print(X[:10])
print(y[:10])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=17)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

[[   5.64    3.92 1065.  ]
 [   4.38    3.4  1050.  ]
 [   5.68    3.17 1185.  ]
 [   7.8     3.45 1480.  ]
 [   4.32    2.93  735.  ]
 [   6.75    2.85 1450.  ]
 [   5.25    3.58 1290.  ]
 [   5.05    3.58 1295.  ]
 [   5.2     2.85 1045.  ]
 [   7.22    3.55 1045.  ]]
[0 0 0 0 0 0 0 0 0 0]
(119, 3) (59, 3) (119,) (59,)


**Задайте тип кросс-валидации с помощью StratifiedKFold: 5-кратная, random_state=17.**

In [10]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

In [11]:
skf = StratifiedKFold(n_splits=5, random_state=17, shuffle=True)

### 2.
Обучите логистическую регрессию (LogisticRegression) с параметром C по умолчанию и random_state=17. Укажите гиперпараметр multi_class='ovr' - по умолчанию многие классификаторы используют именно его. С помощью cross_val_score сделайте кросс-валидацию (используйте объект skf) и выведите среднюю долю правильных ответов на ней (используйте функцию mean). Отдельно выведите долю правильных ответов на тестовой выборке.

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [20]:
lr = LogisticRegression(random_state=17, multi_class="ovr").fit(X_train, y_train)
cross_val = cross_val_score(lr, X_train, y_train, cv=skf)
print(f"Train:\t{cross_val.mean():.3f}")
cross_val = cross_val_score(lr, X_test, y_test, cv=skf)
print(f"Test:\t{cross_val.mean():.3f}")

Train:	0.916
Test:	0.965


### 3.
Обучите метод опорных векторов (SVC) с random_state=17 и остальными параметрами по умолчанию. Этот метод при мультиклассовой классификации также использует метод "ovr". Сделайте кросс-валидацию (используйте skf) и, как и в предыдущем пункте, выведите среднюю долю правильных ответов на ней. Отдельно выведите долю правильных ответов на тестовой выборке.

In [21]:
from sklearn.svm import SVC

In [22]:
svc = SVC(random_state=17).fit(X_train, y_train)
cross_val = cross_val_score(svc, X_train, y_train, cv=skf)
print(f"Train:\t{cross_val.mean():.3f}")
cross_val = cross_val_score(svc, X_test, y_test, cv=skf)
print(f"Test:\t{cross_val.mean():.3f}")

Train:	0.689
Test:	0.764


Как видно из полученной метрики, на тестовой выборке метод с гиперпараметрами по умолчанию работает явно намного хуже логистической регрессии. В целом, SVM достаточно плохо масштабируется на размер обучающего набора данных (как видно, даже с тремя признаками он работает не очень хорошо), но благодаря возможности выбора различных ядер (функций близости, которые помогают разделять данные) и другим гиперпараметрам SVM можно достаточно точно настроить под определенный вид данных. Подробнее на этом останавливаться в контексте данного урока не будем.

### 4.
Для предсказаний обеих моделей постройте матрицу ошибок (confusion matrix) и напишите, какие классы каждая из моделей путает больше всего между собой.

In [23]:
from sklearn.metrics import classification_report, confusion_matrix

In [26]:
y_lr = lr.predict(X_test)
conf_matrix = confusion_matrix(y_test, y_lr)
print(
    "Linear regression confusion matrix:\n"\
    f"{conf_matrix}"
)
y_svc = svc.predict(X_test)
conf_matrix = confusion_matrix(y_test, y_svc)
print(
    "Support vector classification confusion matrix:\n"\
    f"{conf_matrix}"
)

Linear regression confusion matrix:
[[12  0  0]
 [ 0 26  0]
 [ 0  5 16]]
Support vector classification confusion matrix:
[[12  0  0]
 [ 0 26  0]
 [ 3 18  0]]


Linear regression sometimes thinks that 1 class is 2 <br>
SVC thinks that 0 or 1 class is 2 and can't predicts 2 class correctly in valid dataset

### 5.
Для каждой модели выведите classification report.

In [27]:
print(
    "Linear regression report:\n"\
    f"{classification_report(y_test, y_lr)}"
)
print(
    "Support vector classification report:\n"\
    f"{classification_report(y_test, y_svc)}"
)

Linear regression report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       0.84      1.00      0.91        26
           2       1.00      0.76      0.86        21

    accuracy                           0.92        59
   macro avg       0.95      0.92      0.93        59
weighted avg       0.93      0.92      0.91        59

Support vector classification report:
              precision    recall  f1-score   support

           0       0.80      1.00      0.89        12
           1       0.59      1.00      0.74        26
           2       0.00      0.00      0.00        21

    accuracy                           0.64        59
   macro avg       0.46      0.67      0.54        59
weighted avg       0.42      0.64      0.51        59



/home/hik0/mount/500_gb/data_science/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hik0/mount/500_gb/data_science/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hik0/mount/500_gb/data_science/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif